In [54]:
import pandas as pd
import MeCab
import mojimoji
import re
import os
from collections import Counter
from gensim.models import word2vec

from somoclu import Somoclu

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from tqdm import tqdm
import seaborn as sns

%matplotlib inline


In [3]:
word_map_df = pd.read_csv('../output/word_map.csv', index_col=0)

In [65]:
#ユニット用の絡む作成
unit_columns = []
for row in word_map_df['row'].sort_values().unique().tolist():
    for col in word_map_df['col'].sort_values().unique().tolist():
        unit_columns.append(f'{row}_{col}')
len(unit_columns)

162

In [66]:
df = pd.read_csv('../output/preprocessed_data.csv')\
            .dropna(subset=['text'])
df['text'] = df['text'].apply(lambda text: text.split())

In [67]:
df[unit_columns] = 0

In [68]:
text = df['text'].values[0]

In [69]:
for word, row, col in tqdm(zip(word_map_df.index, word_map_df['row'], word_map_df['col'])):
    unit = f'{row}_{col}'
    word_cnt = [text.count(word) for text in df['text']]
    df[unit] += word_cnt

18284it [13:56, 21.86it/s]


In [70]:
df

,url,datetime,title,body,media,text,0_0,0_1,0_2,0_3,...,16_8,17_0,17_1,17_2,17_3,17_4,17_5,17_6,17_7,17_8
0,http://news.livedoor.com/article/detail/5978741/,2011-10-30T10:15:00+0900,【DVDエンター！】誘拐犯に育てられた女が目にした真実は、孤独か幸福か,2005年11月から翌2006年7月まで読売新聞にて連載された、直木賞作家・角田光代による初...,movie-enter,"[年月, 年月, 読売新聞, 連載, する, れる, 直木賞, 作家, 角田光代, 初, 長...",24,4,2,5,...,1,0,3,0,0,5,11,2,0,3
1,http://news.livedoor.com/article/detail/6322901/,2012-02-29T11:45:00+0900,藤原竜也、中学生とともにロケット打ち上げに成功,「アンテナを張りながら生活をしていけばいい」\n2月28日、映画『おかえり、はやぶさ』（3月...,movie-enter,"[*, 張る, 生活, する, いく, いい, 月日, 映画, はやぶさ, 月日, 公開, ...",61,12,12,3,...,0,0,3,7,2,1,0,0,0,1
2,http://news.livedoor.com/article/detail/6176324/,2012-01-09T14:00:00+0900,『戦火の馬』ロイヤル・プレミアにウィリアム王子＆キャサリン妃が出席,3月2日より全国ロードショーとなる、スティーブン・スピルバーグの待望の監督最新作『戦火の馬』...,movie-enter,"[月日, 全国, *, なる, *, *, 待望, 監督, 最新作, 戦火の馬, *, 賞,...",44,12,9,0,...,0,1,8,9,0,6,4,0,0,1
3,http://news.livedoor.com/article/detail/6573929/,2012-05-19T12:00:00+0900,香里奈、女子高生100人のガチンコ質問に回答「ラーメンも食べる」,女優の香里奈が18日、都内で行われた映画『ガール』（5月26日公開）の女子高生限定試写会にサ...,movie-enter,"[女優, 香里奈, 日, 都内, 行う, れる, 映画, *, 月日, 公開, 女子高生, ...",29,10,11,7,...,0,0,2,0,0,0,0,2,0,0
4,http://news.livedoor.com/article/detail/5914880/,2011-10-05T19:11:00+0900,ユージの前に立ちはだかったJOY「僕はAKBの高橋みなみを守る」,5日、東京・千代田区の内幸町ホールにて、映画『キャプテン・アメリカ/ザ・ファースト・アベンジ...,movie-enter,"[日, 東京, 千代田区, 内幸町, *, 映画, *, *, *, *, 公開, 記念, ...",58,19,7,5,...,0,2,1,2,5,5,1,4,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7362,http://news.livedoor.com/article/detail/6530260/,2012-05-05T09:55:00+0900,好きな戦士を作ってドラゴンボールの世界で天下一武道会優勝だ！「挑戦！天下一武道会」【Andr...,どんな戦士を作るかはユーザー次第！\n国民的人気を誇る鳥山明氏のマンガ／アニメである「DRA...,smax,"[戦士, 作る, *, 次第, 国民, 的, 人気, 誇る, 鳥山明, 氏, *, *, 世...",36,16,3,1,...,0,1,0,1,1,0,0,1,1,0
7363,http://news.livedoor.com/article/detail/6681611/,2012-06-21T20:55:00+0900,NTTドコモ、GALAXY SIII SC-06DとF-09D ANTEPRIMAの発売日を...,GALAXY SIIIが6月28日、F-09D ANTEPRIMAが6月27日に発売！\nN...,smax,"[月日, 月日, 発売, *, 日, 今夏, 発売, する, 予定, 年, 夏, *, うち...",1,2,0,0,...,0,0,1,0,0,0,0,0,1,0
7364,http://news.livedoor.com/article/detail/6856578/,2012-08-15T11:55:00+0900,NTTドコモ、Android向け「docomo Wi-Fiかんたん接続アプリ」をバージョンア...,shimajiro@mobiler\nNTTドコモは、同社の公衆無線LANサービス「doco...,smax,"[*, 同社, 公衆, 無線, *, 向け, 設定, *, かんたん, 接続, *, *, ...",31,3,1,0,...,0,1,0,0,0,0,0,0,0,0
7365,http://news.livedoor.com/article/detail/6678539/,2012-06-21T06:55:00+0900,NTTドコモ、PRADA Phone by LG L-02Dのデコメ絵文字popが正常に表示...,PRADA Phone by LG L-02Dにソフトウェア更新！\nNTTドコモは20日、...,smax,"[*, 更新, *, 日, 世界的, *, *, *, する, 高級感, ある, *, 機能...",13,7,5,3,...,0,0,1,1,0,0,0,0,0,5


In [71]:
#df.to_csv('../output/preprocessed_word_map_cnt.csv', index=False)

In [75]:
df = pd.read_csv('../output/preprocessed_word_map_cnt.csv')

In [76]:
df

,url,datetime,title,body,media,text,0_0,0_1,0_2,0_3,...,16_8,17_0,17_1,17_2,17_3,17_4,17_5,17_6,17_7,17_8
0,http://news.livedoor.com/article/detail/5978741/,2011-10-30T10:15:00+0900,【DVDエンター！】誘拐犯に育てられた女が目にした真実は、孤独か幸福か,2005年11月から翌2006年7月まで読売新聞にて連載された、直木賞作家・角田光代による初...,movie-enter,"['年月', '年月', '読売新聞', '連載', 'する', 'れる', '直木賞', ...",24,4,2,5,...,1,0,3,0,0,5,11,2,0,3
1,http://news.livedoor.com/article/detail/6322901/,2012-02-29T11:45:00+0900,藤原竜也、中学生とともにロケット打ち上げに成功,「アンテナを張りながら生活をしていけばいい」\n2月28日、映画『おかえり、はやぶさ』（3月...,movie-enter,"['*', '張る', '生活', 'する', 'いく', 'いい', '月日', '映画'...",61,12,12,3,...,0,0,3,7,2,1,0,0,0,1
2,http://news.livedoor.com/article/detail/6176324/,2012-01-09T14:00:00+0900,『戦火の馬』ロイヤル・プレミアにウィリアム王子＆キャサリン妃が出席,3月2日より全国ロードショーとなる、スティーブン・スピルバーグの待望の監督最新作『戦火の馬』...,movie-enter,"['月日', '全国', '*', 'なる', '*', '*', '待望', '監督', ...",44,12,9,0,...,0,1,8,9,0,6,4,0,0,1
3,http://news.livedoor.com/article/detail/6573929/,2012-05-19T12:00:00+0900,香里奈、女子高生100人のガチンコ質問に回答「ラーメンも食べる」,女優の香里奈が18日、都内で行われた映画『ガール』（5月26日公開）の女子高生限定試写会にサ...,movie-enter,"['女優', '香里奈', '日', '都内', '行う', 'れる', '映画', '*'...",29,10,11,7,...,0,0,2,0,0,0,0,2,0,0
4,http://news.livedoor.com/article/detail/5914880/,2011-10-05T19:11:00+0900,ユージの前に立ちはだかったJOY「僕はAKBの高橋みなみを守る」,5日、東京・千代田区の内幸町ホールにて、映画『キャプテン・アメリカ/ザ・ファースト・アベンジ...,movie-enter,"['日', '東京', '千代田区', '内幸町', '*', '映画', '*', '*'...",58,19,7,5,...,0,2,1,2,5,5,1,4,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7360,http://news.livedoor.com/article/detail/6530260/,2012-05-05T09:55:00+0900,好きな戦士を作ってドラゴンボールの世界で天下一武道会優勝だ！「挑戦！天下一武道会」【Andr...,どんな戦士を作るかはユーザー次第！\n国民的人気を誇る鳥山明氏のマンガ／アニメである「DRA...,smax,"['戦士', '作る', '*', '次第', '国民', '的', '人気', '誇る',...",36,16,3,1,...,0,1,0,1,1,0,0,1,1,0
7361,http://news.livedoor.com/article/detail/6681611/,2012-06-21T20:55:00+0900,NTTドコモ、GALAXY SIII SC-06DとF-09D ANTEPRIMAの発売日を...,GALAXY SIIIが6月28日、F-09D ANTEPRIMAが6月27日に発売！\nN...,smax,"['月日', '月日', '発売', '*', '日', '今夏', '発売', 'する',...",1,2,0,0,...,0,0,1,0,0,0,0,0,1,0
7362,http://news.livedoor.com/article/detail/6856578/,2012-08-15T11:55:00+0900,NTTドコモ、Android向け「docomo Wi-Fiかんたん接続アプリ」をバージョンア...,shimajiro@mobiler\nNTTドコモは、同社の公衆無線LANサービス「doco...,smax,"['*', '同社', '公衆', '無線', '*', '向け', '設定', '*', ...",31,3,1,0,...,0,1,0,0,0,0,0,0,0,0
7363,http://news.livedoor.com/article/detail/6678539/,2012-06-21T06:55:00+0900,NTTドコモ、PRADA Phone by LG L-02Dのデコメ絵文字popが正常に表示...,PRADA Phone by LG L-02Dにソフトウェア更新！\nNTTドコモは20日、...,smax,"['*', '更新', '*', '日', '世界的', '*', '*', '*', 'す...",13,7,5,3,...,0,0,1,1,0,0,0,0,0,5


###  SOMの学習

In [79]:
df.iloc[:, 6:]

,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,0_8,1_0,...,16_8,17_0,17_1,17_2,17_3,17_4,17_5,17_6,17_7,17_8
0,24,4,2,5,2,0,0,0,10,30,...,1,0,3,0,0,5,11,2,0,3
1,61,12,12,3,2,2,7,6,36,31,...,0,0,3,7,2,1,0,0,0,1
2,44,12,9,0,9,7,16,21,47,17,...,0,1,8,9,0,6,4,0,0,1
3,29,10,11,7,2,0,2,1,12,6,...,0,0,2,0,0,0,0,2,0,0
4,58,19,7,5,3,4,6,3,14,21,...,0,2,1,2,5,5,1,4,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7360,36,16,3,1,6,0,0,3,12,15,...,0,1,0,1,1,0,0,1,1,0
7361,1,2,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,1,0
7362,31,3,1,0,0,0,0,1,1,12,...,0,1,0,0,0,0,0,0,0,0
7363,13,7,5,3,1,0,0,0,0,10,...,0,0,1,1,0,0,0,0,0,5


In [109]:
#吉岡の研究では単語マップのノードは162
#n_columns, n_rows  = 100, 100
n_columns, n_rows  = 50, 50
#n_columns, n_rows  = 25, 25
som = Somoclu(n_columns, n_rows,compactsupport=False)
%time som.train(df.iloc[:, 6:].values)

CPU times: user 1min 5s, sys: 573 ms, total: 1min 6s
Wall time: 10.2 s


In [110]:
surface = som.get_surface_state()

In [111]:
doc_map_df = pd.DataFrame(som.get_bmus(surface))
doc_map_df.columns = ['col', 'row']
doc_map_df.index = df.index
doc_map_df = doc_map_df[['row', 'col']]

In [112]:
doc_map_df

,row,col
0,18,2
1,6,14
2,14,49
3,48,30
4,12,47
...,...,...
7360,6,18
7361,49,5
7362,4,31
7363,0,31


In [113]:
df.merge(doc_map_df, how='left', left_index=True, right_index=True).to_csv('../output/preprocessed_data_doc_map.csv', index=False)